In [94]:
import numpy as np
import time
import scipy.io as sio
import copy

In [106]:
def applyPolyCoefComplex(xBatch, coefMatBatch):
    degLen=5
    memLen=3
    memLenM1=memLen-1
    yBatch=np.zeros_like(xBatch)
    for batchIdx in range(xBatch.shape[0]):
        x=xBatch[batchIdx]
        coefMat = coefMatBatch[batchIdx]
        coefMatTrans=coefMat.transpose()
        coeftMatReshaped = coefMatTrans.reshape((-1))
        xLen=x.shape[0]
        # y=copy.deepcopy(x) #np.zeros_like(x)
        y=np.zeros_like(x)
        for timeIdx in range(memLen, xLen+1):
            xTerms=np.zeros((memLen*degLen), dtype=np.complex_)
            xTime=x[timeIdx-memLen: timeIdx]
            xTime=np.flipud(xTime)
            xTerms[:memLen] = xTime[:]
            for degIdx in range(1, degLen):
                startPos = degIdx*memLen
                endPos = startPos+memLen
                tmp=np.abs(xTime)**degIdx
                xTerms[startPos:endPos] = xTime[:]*tmp[:]
            y_tmp=coeftMatReshaped*xTerms
            y[timeIdx-1] = sum(y_tmp)
        yBatch[batchIdx]=y
    return yBatch

In [151]:
def fastPolyComp(xBatch, coefMatBatch):
    degLen=5
    memLen=3
    xLen=xBatch.shape[-1]
    time_steps = xLen+1 - memLen
    batch_size = xBatch.shape[0]
    x_extented = np.zeros((batch_size, time_steps, memLen), dtype=np.complex_)
    coefmat = coefMatBatch.transpose(0, 2, 1).reshape(batch_size, -1) #(batch_size, memLen, degLen)->(batch_size, degLen, memLen)->(batch_size, degLen*memLen)
    yBatch= np.zeros_like(xBatch, dtype=np.complex_) #copy.deepcopy(xBatch) #
    for t in range(time_steps):
        x_extented[:, t] = np.fliplr(xBatch[:, t: t+memLen])
    x_extented_abs = np.abs(x_extented)
    xTerms = np.stack([x_extented_abs for _ in range(degLen)], axis=0) # (degLen, batch_size, time_steps, memLen)
    degs = np.zeros_like(xTerms)
    for d in range(degLen):
        degs[d] = d*np.ones_like(xTerms[d])  # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms ** degs # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms[:]*x_extented # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms.transpose(1, 2, 0, 3).reshape(batch_size, time_steps, -1).transpose(1, 0, 2) # (degLen, batch_size, time_steps, memLen)->(batch_size, time_steps, degLen*memLen)->(time_steps, batch_size, degLen*memLen)
    y_tmp = xTerms[:]*coefmat
    y_tmp = np.sum(y_tmp, axis=-1).transpose().reshape(batch_size, time_steps)
    yBatch[:, memLen-1:] = y_tmp
    return yBatch

def fastPolyCompPrefix(xBatch):
    degLen=5
    memLen=3
    xLen=xBatch.shape[-1]
    time_steps = xLen+1 - memLen
    batch_size = xBatch.shape[0]
    x_extented = np.zeros((batch_size, time_steps, memLen), dtype=np.complex_)
    
    for t in range(time_steps):
        x_extented[:, t] = np.fliplr(xBatch[:, t: t+memLen])
    x_extented_abs = np.abs(x_extented)
    xTerms = np.stack([x_extented_abs for _ in range(degLen)], axis=0) # (degLen, batch_size, time_steps, memLen)
    degs = np.zeros_like(xTerms)
    for d in range(degLen):
        degs[d] = d*np.ones_like(xTerms[d])  # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms ** degs # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms[:]*x_extented # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms.transpose(1, 2, 0, 3).reshape(batch_size, time_steps, -1).transpose(1, 0, 2) # (degLen, batch_size, time_steps, memLen)->(batch_size, time_steps, degLen*memLen)->(time_steps, batch_size, degLen*memLen)
    return xTerms

def fastPolyCompPrefixOut(xBatch, xTerms, coefMatBatch):
    time_steps, batch_size, dm = xTerms.shape
    coefmat = coefMatBatch.transpose(0, 2, 1).reshape(batch_size, -1)
    # yBatch= copy.deepcopy(xBatch) #np.zeros_like(xBatch, dtype=np.complex_)
    yBatch = np.zeros_like(xBatch,dtype=np.complex_)
    y_tmp = xTerms[:]*coefmat
    y_tmp = np.sum(y_tmp, axis=-1).transpose().reshape(batch_size, time_steps)
    yBatch[:, 3-1:] = y_tmp
    return yBatch

# x = sio.loadmat("./x.mat")['x']
# coefmat = sio.loadmat("./coef.mat")['coef']
x = np.random.randn(16, 100)+1j*np.random.randn(16, 100)
coefmat = np.random.randn(16, 3, 5)

n=100
start_fast=time.time()
for _ in range(n):
    y_fast = fastPolyComp(x, coefmat)  
end_fast=time.time()

start_pre=time.time()
pre = fastPolyCompPrefix(x)
for _ in range(n):
    y_pre = fastPolyCompPrefixOut(x, pre, coefmat)
end_pre=time.time()

start=time.time()
for _ in range(n):
    y = applyPolyCoefComplex(x, coefmat)
end=time.time()

err1 = np.sum((np.abs(y_fast-y)))
err2 = np.sum((np.abs(y_pre-y)))
err3 = np.sum((np.abs(y_fast-y_pre)))
print("time_fast:{}, time_pre:{}, time_normal:{}".format((end_fast-start_fast)/n, (end_pre-start_pre)/n, (end-start)/n))
print("error fastVSnormal:{}, error preVSnormal:{}, error fastVSpre:{}".format(err1, err2, err3))

time_fast:0.0007100081443786621, time_pre:8.000373840332031e-05, time_normal:0.02596620798110962
error fastVSnormal:2.7431340692379824e-12, error preVSnormal:2.7431340692379824e-12, error fastVSpre:0.0


In [157]:
def fastPoly(xBatch, coefMatBatch): #(batch_size, 2, 800)
    degLen=5
    memLen=3
    xLen=xBatch.shape[-1]
    time_steps = xLen+1 - memLen
    batch_size = xBatch.shape[0]
    x_extented = np.zeros((batch_size, 2, time_steps, memLen))
    coefmat = coefMatBatch.transpose(0, 2, 1).reshape(batch_size, -1) #(batch_size, memLen, degLen)->(batch_size, degLen, memLen)->(batch_size, degLen*memLen)
    yBatch= np.zeros_like(xBatch) #(batch_size, 2, 800)
    for t in range(time_steps):
        x_extented[:, :, t] = np.flip(xBatch[:, :, t: t+memLen], axis=-1)
    x_extented_abs = np.power(x_extented, 2) 
    x_extented_abs = np.sqrt(x_extented_abs[:, 0, :, :] + x_extented_abs[:, 1, :, :]) #(batch_size, time_steps, memLen)
    xTerms = np.stack([x_extented_abs for _ in range(degLen)], axis=0) # (degLen, batch_size, time_steps, memLen)
    degs = np.zeros_like(xTerms)
    for d in range(degLen):
        degs[d] = d*np.ones_like(xTerms[d])  # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms ** degs # (degLen, batch_size, time_steps, memLen)
    xTerms_real= xTerms[:]*x_extented[:, 0, :, :] # (degLen, batch_size, time_steps, memLen)
    xTerms_imag= xTerms[:]*x_extented[:, 1, :, :]
    xTerms = np.stack([xTerms_real, xTerms_imag], axis=2)
    xTerms = xTerms.transpose(1, 2, 3, 0, 4).reshape(batch_size, 2, time_steps, -1).transpose(1, 2, 0, 3) # (degLen, batch_size, time_steps, memLen)->(batch_size, time_steps, degLen*memLen)->(time_steps, batch_size, degLen*memLen)
    y_tmp = xTerms[:,:]*coefmat
    y_tmp = np.sum(y_tmp, axis=-1).transpose(2, 0, 1)#.reshape(batch_size, 2, time_steps)
    yBatch[:, :, memLen-1:] = y_tmp
    return yBatch

def fastPolyPrefix(xBatch): #(batch_size, 2, 800)
    degLen=5
    memLen=3
    xLen=xBatch.shape[-1]
    time_steps = xLen+1 - memLen
    batch_size = xBatch.shape[0]
    x_extented = np.zeros((batch_size, 2, time_steps, memLen))
    for t in range(time_steps):
        x_extented[:, :, t] = np.flip(xBatch[:, :, t: t+memLen], axis=-1)
    x_extented_abs = np.power(x_extented, 2) 
    x_extented_abs = np.sqrt(x_extented_abs[:, 0, :, :] + x_extented_abs[:, 1, :, :]) #(batch_size, time_steps, memLen)
    xTerms = np.stack([x_extented_abs for _ in range(degLen)], axis=0) # (degLen, batch_size, time_steps, memLen)
    degs = np.zeros_like(xTerms)
    for d in range(degLen):
        degs[d] = d*np.ones_like(xTerms[d])  # (degLen, batch_size, time_steps, memLen)
    xTerms = xTerms ** degs # (degLen, batch_size, time_steps, memLen)
    xTerms_real= xTerms[:]*x_extented[:, 0, :, :] # (degLen, batch_size, time_steps, memLen)
    xTerms_imag= xTerms[:]*x_extented[:, 1, :, :]
    xTerms = np.stack([xTerms_real, xTerms_imag], axis=2) # (degLen, batch_size, 2, time_steps, memLen)
    # (degLen, batch_size, 2, time_steps, memLen)->(batch_size, 2, time_steps, degLen*memLen)->(2, time_steps, batch_size, degLen*memLen)
    xTerms = xTerms.transpose(1, 2, 3, 0, 4).reshape(batch_size, 2, time_steps, -1).transpose(1, 2, 0, 3)     
    return xTerms

def fastPolyPrefixOut(xBatch, xTerms, coefMatBatch, memLen):
    _, time_steps, batch_size, dm = xTerms.shape #(2, time_steps, batch_size, degLen*memLen)
    coefmat = coefMatBatch.transpose(0, 2, 1).reshape(batch_size, -1)
    yBatch = np.zeros_like(xBatch)
    y_tmp = xTerms[:,:]*coefmat
    y_tmp = np.sum(y_tmp, axis=-1).transpose(2, 0, 1)#.reshape(batch_size, 2, time_steps)
    yBatch[:, :, memLen-1:] = y_tmp
    return yBatch

x = sio.loadmat("./x.mat")['x']
coefmat = sio.loadmat("./coef.mat")['coef']

x_real = x.real
x_imag = x.imag
x_2h = np.stack([x_real, x_imag], axis=1)
# real = np.random.randn(16, 100)
# imag = np.random.randn(16, 100)
# x = real+1j*imag
# x_real = np.stack([real, imag], axis=1)
# coefmat = np.random.randn(16, 3, 5)

n=100
start_fast=time.time()
for _ in range(n):
    y_fast = fastPolyComp(x, coefmat)  
end_fast=time.time() 

start_2ch=time.time()
for _ in range(n):
    y_2ch = fastPoly(x_2h, coefmat)
    y_2ch = y_2ch[:,0,:]+1j*y_2ch[:,1,:]
end_2ch=time.time()

start_2ch_pre=time.time()
y_2ch_pre = fastPolyPrefix(x_2h, coefmat)
for _ in range(n):   
    y_2ch_pre_out = fastPolyPrefixOut(x_2h, y_2ch_pre, coefmat, 3)
    y_2ch_pre_out = y_2ch_pre_out[:,0,:]+1j*y_2ch_pre_out[:,1,:]
end_2ch_pre=time.time()

start=time.time()
for _ in range(n):
    y = applyPolyCoefComplex(x, coefmat)
end=time.time()

err1 = np.sum((np.abs(y_fast-y)))
err2 = np.sum((np.abs(y_2ch-y)))
err2 = np.sum((np.abs(y_2ch_pre_out-y)))
err3 = np.sum((np.abs(y_2ch-y_2ch_pre_out)))
print("time_fast:{}, time_2ch:{}, time_2ch_pre:{}, time_normal:{}".format((end_fast-start_fast)/n, (end_2ch-start_2ch)/n, (end_2ch_pre-start_2ch_pre)/n,(end-start)/n))
print("error fastVSnormal:{}, error preVSnormal:{}, error fastVSpre:{}".format(err1, err2, err3))

time_fast:0.001150026321411133, time_2ch:0.0012400054931640625, time_2ch_pre:9.999752044677735e-05, time_normal:0.024855437278747557
error fastVSnormal:6.733166315688891e-13, error preVSnormal:3.748198429000568e-13, error fastVSpre:0.0
